In [1]:
import numpy as np
import pandas as pd
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)

In [2]:
race_df = pd.read_csv("./csv/race-2008.csv", sep=",")
horse_df = pd.read_csv("./csv/horse-2008.csv", sep=",")
for year in range(2009, 2019):
    race_tmp_df = pd.read_csv("./csv/race-"+str(year)+".csv", sep=",")
    horse_tmp_df = pd.read_csv("./csv/horse-"+str(year)+".csv", sep=",")
    race_df = pd.concat([race_df, race_tmp_df], axis=0)
    horse_df = pd.concat([horse_df, horse_tmp_df], axis=0)

/usr/local/var/pyenv/versions/3.6.5/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 元データの確認

In [3]:
# race_id単位で重複したデータが存在しないか確認
print(len(race_df) == len(race_df['race_id'].unique()))
print(race_df.shape)
race_df.tail(2)

True
(37988, 28)


,race_id,race_round,race_title,race_course,weather,ground_status,time,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3
3452,201809050510,10 R,赤穂特別,ダ右1800m,天候 : 晴,ダート : 良,発走 : 15:10\n\n過去の赤穂特別,2018年12月15日,5回阪神5日目,14,7,11,8,14,3,4,640,220,"1,210",360,"1,960","14,150","4,640","1,740","9,500","25,610","73,670","346,360"
3453,201807040407,7 R,2歳未勝利,芝左2000m,天候 : 晴,芝 : 良,発走 : 13:15,2018年12月9日,4回中京4日目,14,8,13,1,1,6,9,"1,450",330,160,350,"1,440","3,150",810,"2,060",990,"8,190","10,070","64,560"


In [4]:
# 出走馬数の確認
race_df["total_horse_number"].value_counts()

16    14169
15     4367
14     3834
18     3549
13     3082
12     2828
11     1997
10     1576
9      1057
17      628
8       602
7       212
6        69
5        18
Name: total_horse_number, dtype: int64

In [5]:
print(horse_df.shape)
horse_df.head(2)

(548489, 19)


,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,time_value,half_way_rank,last_time,odds,popular,horse_weight,tame_time,tamer_id,owner_id
0,200810010312,1,4,7,2004110053,牡4,56.0,674,1:09.7,NaN,\n\n**\n\n,11-12,35.9,9.7,4.0,484(-10),\n\n,1075,19004
1,200810010312,2,3,6,2003106038,牡5,57.0,1054,1:09.7,クビ,\n\n**\n\n,7-8,36.1,35.7,12.0,474(-8),\n\n,1007,910006


## raceデータの整形

In [6]:
race_df.head(1)

,race_id,race_round,race_title,race_course,weather,ground_status,time,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3
0,200810010312,12 R,鳥栖特別,芝右1200m,天候 : 晴,芝 : 良,発走 : 16:05\n\n過去の鳥栖特別,2008年1月26日,1回小倉3日目,18,4,7,3,6,1,2,970,280,820,"1,250","2,790","13,670","4,400","8,080","10,850","29,590","152,100","836,940"


### race_id
そのままでOK

In [7]:
# 一応確認
race_df["race_id"].dtypes

dtype('int64')

### race_round
余分な空白とRを取り除く

In [8]:
race_df["race_round"].dtypes

dtype('O')

In [9]:
race_df['race_round'].unique()

array(['12 R', '9 R', '7 R', '3 R', '5 R', '6 R', '11 R', '4 R', '10 R',
       '8 R', '1 R', '2 R', '      1 R\n      '], dtype=object)

In [10]:
race_df['race_round'] = race_df['race_round'].str.strip('R \n')

In [11]:
race_df['race_round'].unique()

array(['12', '9', '7', '3', '5', '6', '11', '4', '10', '8', '1', '2'],
      dtype=object)

In [12]:
race_df['race_round'] = race_df['race_round'].astype(int)
race_df["race_round"].dtypes

dtype('int64')

### race_title
いらないので削除

In [13]:
# もともとのカラムは不要なので削除
race_df.drop(['race_title'], axis=1, inplace=True)

### race_course
「ダ右1200m」などであれば、ダート・右回り・1200に分割して、それぞれ別のカラムにする。

新たに4つのカラムを追加
- 障害コースか？
- 地面のタイプは何か？
- 右回り・左回り・直線か？
- 距離は？

In [14]:
race_df["race_course"].unique()

array(['芝右1200m', '芝右1800m', 'ダ右1200m', 'ダ右1400m', '芝右 外2200m', 'ダ右1000m',
       '芝右 外1600m', 'ダ右1800m', 'ダ右1700m', '芝右2000m', '芝右 外1800m',
       '芝右1600m', '障芝 ダート2930m', '芝右 外1400m', '障芝 ダート3200m',
       '障芝 ダート2880m', '芝右 外1200m', '芝右2600m', '障芝 ダート3190m', '芝右 外2400m',
       '芝右2500m', 'ダ右2400m', '芝右 外3000m', 'ダ右2500m', '芝右1400m', 'ダ左1600m',
       'ダ左1400m', '芝左1400m', '芝左1600m', '芝左3400m', '芝左2000m', 'ダ左2100m',
       'ダ左1300m', '障芝 ダート3790m', '障芝 ダート3000m', 'ダ左2400m', '芝左1800m',
       '芝左2400m', 'ダ右1900m', '障芝 ダート3300m', 'ダ左1700m', '芝左1200m',
       '障芝 ダート2970m', '芝右3000m', 'ダ左1000m', '障芝 外3350m', '芝右2200m',
       'ダ右2000m', 'ダ左2300m', '障芝 ダート3110m', '芝左2500m', '障芝3900m',
       '障芝 ダート3170m', 'ダ右1150m', '芝左2300m', '障芝 ダート2910m', '障芝 外4250m',
       '障芝3140m', 'ダ左1200m', '芝直線1000m', '芝左 外1600m', '芝左 外1800m',
       'ダ左1800m', '芝左2200m', '芝左 外2000m', '障芝 ダート3100m', '芝右 外3200m',
       '障芝 左2800m', '障芝 内-外3170m', '障芝2750m', '芝右1700m', '障芝 左3370m',
       '芝右1000m', '障芝3300m'

In [15]:
# 正規表現で取得

# 障害か、地面のタイプは何か、左か、右か、直線か、
obstacle = race_df["race_course"].str.extract('(障)', expand=True)
ground_type = race_df["race_course"].str.extract('(ダ|芝)', expand=True)
is_left_right_straight = race_df["race_course"].str.extract('(左|右|直線)', expand=True)
distance = race_df["race_course"].str.extract('(\d+)m', expand=True)

obstacle.columns ={"is_obstacle"}
ground_type.columns ={"ground_type"}
is_left_right_straight.columns = {"is_left_right_straight"}
distance.columns = {"distance"}

race_df = pd.concat([race_df, obstacle], axis=1)
race_df = pd.concat([race_df, ground_type], axis=1)
race_df = pd.concat([race_df, is_left_right_straight], axis=1)
race_df = pd.concat([race_df, distance], axis=1)

In [16]:
# 'is_obstacle' 列の '障芝' を1に置き換え、Nanに0埋め
race_df['is_obstacle'] = race_df['is_obstacle'].replace('障', 1)
race_df.fillna(value={'is_obstacle': 0}, inplace=True)

In [17]:
print("is_obstacle:", race_df["is_obstacle"].unique())
print("ground_type:", race_df["ground_type"].unique())
print("is_left_right_straight:", race_df["is_left_right_straight"].unique())
print("distance isnull sum:", race_df["distance"].isnull().sum())

is_obstacle: [0. 1.]
ground_type: ['芝' 'ダ']
is_left_right_straight: ['右' nan '左' '直線']
distance isnull sum: 0


In [18]:
# もともとのカラムは不要なので削除
race_df.drop(['race_course'], axis=1, inplace=True)

In [19]:
race_df["distance"] = race_df["distance"].astype(int)

### weather
そのままone_hotエンコーディングしてデータを食わせても良さそうだが...

余分な文字列を取り除く。

また、少雨よりも雨が強いはず、小雪よりも雪が強いはず。これらの単純な雨量は別のデータを取ってこないと分からないが、大小関係は情報として入れられるはず。

In [20]:
race_df["weather"].unique()

array(['天候 : 晴', '天候 : 曇', '天候 : 小雨', '天候 : 雨', '天候 : 小雪', '天候 : 雪'],
      dtype=object)

In [21]:
race_df['weather'] = race_df['weather'].str.strip('天候 :')

In [22]:
race_df["weather"].unique()

array(['晴', '曇', '小雨', '雨', '小雪', '雪'], dtype=object)

In [23]:
weather_rain = race_df["weather"].str.extract('(小雨|雨)', expand=True)
weather_snow = race_df["weather"].str.extract('(小雪|雪)', expand=True)
weather_rain.columns ={"weather_rain"}
weather_snow.columns ={"weather_snow"}
race_df = pd.concat([race_df, weather_rain], axis=1)
race_df = pd.concat([race_df, weather_snow], axis=1)

race_df.fillna(value={'weather_rain': 0}, inplace=True)
race_df['weather_rain'] = race_df['weather_rain'].replace('小雨', 1)
race_df['weather_rain'] = race_df['weather_rain'].replace('雨', 2)
race_df.fillna(value={'weather_snow': 0}, inplace=True)
race_df['weather_snow'] = race_df['weather_snow'].replace('小雪', 1)
race_df['weather_snow'] = race_df['weather_snow'].replace('雪', 2)

In [24]:
print("weather_rain:", race_df["weather_rain"].value_counts())
print("weather_snow:", race_df["weather_snow"].value_counts())

weather_rain: 0    34724
2     1954
1     1310
Name: weather_rain, dtype: int64
weather_snow: 0    37922
1       48
2       18
Name: weather_snow, dtype: int64


### ground_status
芝かダートかは既に別カラムにあるので、状態を見る。
大小関係があるので数値として。

In [25]:
race_df["ground_status"].unique()

array(['芝 : 良', 'ダート : 稍重', 'ダート : 重', 'ダート : 良', '芝 : 稍重', '芝 : 重',
       '芝 : 良\xa0\xa0ダート : 稍重', '芝 : 不良', '芝 : 良\xa0\xa0ダート : 良',
       'ダート : 不良', '芝 : 稍重\xa0\xa0ダート : 重', '芝 : 良\xa0\xa0ダート : 不良',
       '芝 : 重\xa0\xa0ダート : 不良', '芝 : 稍重\xa0\xa0ダート : 稍重',
       '芝 : 稍重\xa0\xa0ダート : 不良', '芝 : 重\xa0\xa0ダート : 重',
       '芝 : 良\xa0\xa0ダート : 重', '芝 : 不良\xa0\xa0ダート : 不良'], dtype=object)

In [26]:
race_df['ground_status'] = race_df['ground_status'].replace('.*(稍重).*', 4,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(重).*', 3,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(不良).*', 2,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(良).*', 1,regex=True)


In [27]:
print("ground_status:", race_df["ground_status"].value_counts())

ground_status: 1    27407
4     5874
3     3087
2     1620
Name: ground_status, dtype: int64


### time と dateをあわせてdatetimeに

In [28]:
race_df["time"] = race_df["time"].str.replace('発走 : (\d\d):(\d\d)(.|\n)*', r'\1時\2分')

In [29]:
race_df["date"] = race_df["date"] + race_df["time"]

In [30]:
race_df["date"] = pd.to_datetime(race_df['date'], format='%Y年%m月%d日%H時%M分')

In [31]:
# もともとのtimeは不要なので削除
race_df.drop(['time'], axis=1, inplace=True)

In [32]:
print(race_df["date"].dtype)
print("date isnull sum:", race_df["date"].isnull().sum())

datetime64[ns]
date isnull sum: 0


### where_racecourse
例:1回小倉3日目 の中から小倉を取り出す

In [33]:
race_df["where_racecourse"] = race_df["where_racecourse"].str.replace('\d*回(..)\d*日目', r'\1')


In [34]:
# 確認
race_df["where_racecourse"].unique()

array(['小倉', '中山', '京都', '東京', '阪神', '中京', '福島', '新潟', '函館', '札幌'],
      dtype=object)

###  馬の数や順位
- total_horse_number                 int64
- frame_number_first                 int64
- horse_number_first                 int64
- frame_number_second                int64
- horse_number_second                int64
- frame_number_third                 int64
- horse_number_third                 int64

これらはそのままでOK

### オッズから余分な「,」を除く
- tansyo                            object
- hukuren_first                     object
- hukuren_second                    object
- hukuren_third                     object
- renhuku3                          object
- rentan3                           object

数値と文字列が混在しているので面倒
```
race_df['tansyo'] = race_df['tansyo'].str.strip(',')
```
などとしてもだめ

In [36]:
race_df.columns

Index(['race_id', 'race_round', 'weather', 'ground_status', 'date',
       'where_racecourse', 'total_horse_number', 'frame_number_first',
       'horse_number_first', 'frame_number_second', 'horse_number_second',
       'frame_number_third', 'horse_number_third', 'tansyo', 'hukusyo_first',
       'hukusyo_second', 'hukusyo_third', 'wakuren', 'umaren', 'wide_1_2',
       'wide_1_3', 'wide_2_3', 'umatan', 'renhuku3', 'rentan3', 'is_obstacle',
       'ground_type', 'is_left_right_straight', 'distance', 'weather_rain',
       'weather_snow'],
      dtype='object')

In [38]:
race_df['tansyo'] = race_df['tansyo'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['hukusyo_first'] = race_df['hukusyo_first'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['hukusyo_second'] = race_df['hukusyo_second'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['hukusyo_third'] = race_df['hukusyo_third'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['wakuren'] = race_df['wakuren'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['umaren'] = race_df['umaren'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['wide_1_2'] = race_df['wide_1_2'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['wide_1_3'] = race_df['wide_1_3'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['wide_2_3'] = race_df['wide_2_3'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['umatan'] = race_df['umatan'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['renhuku3'] = race_df['renhuku3'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))
race_df['rentan3'] = race_df['rentan3'].apply(lambda x: int(x.replace(",", "")) if type(x) is str else int(x))


In [39]:
# 確認
race_df['race_id'] = race_df['race_id'].astype(str)
#race_df['race_title'] = race_df['race_title'].astype(str)
print('dataframeの各列のデータ型を確認==>\n', race_df.dtypes)


race_df.head(1)

dataframeの各列のデータ型を確認==>
 race_id                           object
race_round                         int64
weather                           object
ground_status                      int64
date                      datetime64[ns]
where_racecourse                  object
total_horse_number                 int64
frame_number_first                 int64
horse_number_first                 int64
frame_number_second                int64
horse_number_second                int64
frame_number_third                 int64
horse_number_third                 int64
tansyo                             int64
hukusyo_first                      int64
hukusyo_second                     int64
hukusyo_third                      int64
wakuren                            int64
umaren                             int64
wide_1_2                           int64
wide_1_3                           int64
wide_2_3                           int64
umatan                             int64
renhuku3                        

,race_id,race_round,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,is_obstacle,ground_type,is_left_right_straight,distance,weather_rain,weather_snow
0,200810010312,12,晴,1,2008-01-26 16:05:00,小倉,18,4,7,3,6,1,2,970,280,820,1250,2790,13670,4400,8080,10850,29590,152100,836940,0.0,芝,右,1200,0,0


## race dataの保存

In [40]:
race_df.describe()

,race_round,ground_status,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,is_obstacle,distance,weather_rain,weather_snow
count,37988.000000,37988.000000,37988.000000,37988.000000,37988.000000,37988.000000,37988.000000,37988.000000,37988.000000,3.798800e+04,37988.000000,37988.000000,37988.000000,3.798800e+04,3.798800e+04,37988.000000,37988.000000,37988.000000,3.798800e+04,3.798800e+04,3.798800e+04,37988.000000,37988.000000,37988.000000,37988.000000
mean,6.497315,1.669053,14.438481,4.766453,7.723544,4.758003,7.701116,4.686006,7.570075,1.927910e+03,286.935874,360.495683,432.277825,7.766321e+04,5.291406e+05,1571.695799,1908.628778,2466.295672,6.463057e+06,5.110043e+06,7.988270e+10,0.037328,1663.317758,0.137359,0.002211
std,3.450795,1.145315,2.503675,2.267196,4.389452,2.262945,4.384952,2.273300,4.401759,5.069402e+04,395.158785,502.977722,599.304644,8.209003e+06,4.988059e+07,3174.956133,3730.988043,4663.272063,1.065520e+09,5.383521e+08,1.530683e+13,0.189566,440.683434,0.470502,0.056161
min,1.000000,1.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.100000e+02,100.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,110.000000,110.000000,110.000000,1.700000e+02,1.300000e+02,0.000000e+00,0.000000,1000.000000,0.000000,0.000000
25%,3.000000,1.000000,13.000000,3.000000,4.000000,3.000000,4.000000,3.000000,4.000000,2.900000e+02,140.000000,150.000000,160.000000,5.900000e+02,8.700000e+02,360.000000,440.000000,530.000000,1.670000e+03,2.130000e+03,9.670000e+03,0.000000,1300.000000,0.000000,0.000000
50%,6.000000,1.000000,15.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000e+02,180.000000,210.000000,240.000000,1.090000e+03,1.910000e+03,690.000000,850.000000,1060.000000,3.710000e+03,5.495000e+03,2.872000e+04,0.000000,1700.000000,0.000000,0.000000
75%,9.000000,2.000000,16.000000,7.000000,11.000000,7.000000,11.000000,7.000000,11.000000,1.000000e+03,280.000000,360.000000,450.000000,2.290000e+03,4.900000e+03,1520.000000,1860.000000,2410.000000,9.780000e+03,1.655250e+04,9.473000e+04,0.000000,1800.000000,0.000000,0.000000
max,12.000000,4.000000,18.000000,8.000000,18.000000,8.000000,18.000000,8.000000,18.000000,6.603380e+06,11120.000000,14080.000000,16110.000000,1.255010e+09,7.881099e+09,124860.000000,129000.000000,109100.000000,2.073503e+11,9.832032e+10,2.983295e+15,1.000000,4260.000000,2.000000,2.000000


In [41]:
race_df.head()

,race_id,race_round,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,is_obstacle,ground_type,is_left_right_straight,distance,weather_rain,weather_snow
0,200810010312,12,晴,1,2008-01-26 16:05:00,小倉,18,4,7,3,6,1,2,970,280,820,1250,2790,13670,4400,8080,10850,29590,152100,836940,0.0,芝,右,1200,0,0
1,200806010709,9,晴,1,2008-01-26 14:30:00,中山,14,8,14,3,3,5,8,1040,290,220,110,5460,6560,1570,400,320,14450,2750,33860,0.0,芝,右,1800,0,0
2,200808010707,7,曇,4,2008-01-26 13:30:00,京都,14,8,13,3,3,6,9,430,190,1670,2460,5430,18350,4950,4930,35620,28300,237760,0,0.0,ダ,右,1200,0,0
3,200808010303,3,小雨,3,2008-01-12 11:05:00,京都,16,4,7,5,10,1,2,1420,380,460,520,1980,7320,2100,3240,3320,14320,39220,0,0.0,ダ,右,1400,1,0
4,200808010605,5,曇,1,2008-01-20 12:30:00,京都,16,1,2,8,16,1,1,880,300,810,160,3740,16100,4460,620,1710,30570,14710,0,0.0,芝,右,2200,0,0


In [42]:
race_df.to_csv("csv/cleaned_race_data.csv", index=False )

## horse data の整形

In [43]:
print(horse_df.shape)
print(horse_df.dtypes)
horse_df['race_id'] = horse_df['race_id'].astype(str)
horse_df['horse_id'] = horse_df['horse_id'].astype(str)
horse_df['tamer_id'] = horse_df['tamer_id'].astype(str)
horse_df['owner_id'] = horse_df['owner_id'].astype(str)
horse_df['rider_id'] = horse_df['rider_id'].astype(str)

horse_df.head(2)

(548489, 19)
race_id            int64
rank              object
frame_number       int64
horse_number       int64
horse_id           int64
sex_and_age       object
burden_weight    float64
rider_id           int64
goal_time         object
goal_time_dif     object
time_value        object
half_way_rank     object
last_time        float64
odds              object
popular          float64
horse_weight      object
tame_time         object
tamer_id           int64
owner_id          object
dtype: object


,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,time_value,half_way_rank,last_time,odds,popular,horse_weight,tame_time,tamer_id,owner_id
0,200810010312,1,4,7,2004110053,牡4,56.0,674,1:09.7,NaN,\n\n**\n\n,11-12,35.9,9.7,4.0,484(-10),\n\n,1075,19004
1,200810010312,2,3,6,2003106038,牡5,57.0,1054,1:09.7,クビ,\n\n**\n\n,7-8,36.1,35.7,12.0,474(-8),\n\n,1007,910006


In [44]:
# 何かとデータ分析で便利なので、レース日時情報をmerge
race_tmp_df = race_df[["race_id", "date"]]
horse_df = pd.merge(horse_df, race_tmp_df, on='race_id')
horse_df.head()

,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,time_value,half_way_rank,last_time,odds,popular,horse_weight,tame_time,tamer_id,owner_id,date
0,200810010312,1,4,7,2004110053,牡4,56.0,674,1:09.7,NaN,\n\n**\n\n,11-12,35.9,9.7,4.0,484(-10),\n\n,1075,19004,2008-01-26 16:05:00
1,200810010312,2,3,6,2003106038,牡5,57.0,1054,1:09.7,クビ,\n\n**\n\n,7-8,36.1,35.7,12.0,474(-8),\n\n,1007,910006,2008-01-26 16:05:00
2,200810010312,3,1,2,2004110268,牝4,53.0,5223,1:09.9,1.1/4,\n\n**\n\n,7-8,36.3,54.5,14.0,442(-18),\n\n,427,975009,2008-01-26 16:05:00
3,200810010312,4,7,15,2003103747,牝5,55.0,1099,1:09.9,ハナ,\n\n**\n\n,13-12,35.8,25.9,10.0,458(0),\n\n,1041,102800,2008-01-26 16:05:00
4,200810010312,5,6,12,2004101251,牝4,54.0,1075,1:09.9,クビ,\n\n**\n\n,6-3,36.4,49.2,13.0,456(-4),\n\n,437,851005,2008-01-26 16:05:00


### 使わなさそうな情報を削除
- time_value, tame_time(プレミアム会員向けの情報)
- goal_time_dif(自分で作成する)

In [45]:
horse_df.drop(['time_value'], axis=1, inplace=True)
horse_df.drop(['goal_time_dif'], axis=1, inplace=True)
horse_df.drop(['tame_time'], axis=1, inplace=True)

### race_id
そのままでOK

### rank
> - 降着・・・	「その走行妨害がなければ被害馬が加害馬に先着していた」と判断した場合、加害馬は被害馬の後ろに降着となります。
> - 失格・・・	「極めて悪質で他の騎手や馬に対する危険な行為によって、競走に重大な支障を生じさせた」と判断した場合、加害馬は失格となります。

> 注記：被害馬が落馬や疾病発症等により競走を中止した場合には、上記の「失格」に該当しない限り着順は到達順位のとおり確定します。

- 中(中止。オッズはある。ゴールに到達しない...？)         2752
- 取(取り消し。オッズもない。そもそも参加していない．．．？)         1069
- 除(除名？。オッズもない。そもそも参加していない．．．？)          889

は結構な頻度で起きる。他の異常値は少ないので削除するかまとめる...？

- 降格は降格フラグに分割、順位そのまま入れておく
- 取・除はそもそも参加していないので削除
- 失は順位が全く当てにならないので情報を削除
- 中は最後まで到達していないが参加はしている。ひとまず20位にしておく。goal_timeが無いので、大きめに取る必要がある。
- 12(再)は12で最後の模様。そのまま12にする

In [46]:
# 確認
horse_df[horse_df['rank'] =='中'].sort_values('date').head(2)
horse_df[horse_df['rank'] =='取'].sort_values('date').head(2)
horse_df[horse_df['rank'] =='除'].sort_values('date').head(2)
horse_df[horse_df['rank'] =='16(降)'].sort_values('date').head(2)
horse_df[horse_df['rank'] =='12(再)'].sort_values('date').head(2)

,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date
204608,201209010104,12(再),5,6,2006101490,牡6,60.0,1072,3:55.2,9-9-10-12,15.8,45.6,9.0,502(-2),1110,090006,2012-02-25 11:30:00


In [47]:
# 降格を別へ
is_down = horse_df["rank"].str.extract('(\(降\))', expand=True)
is_down.columns ={"is_down"}
horse_df = pd.concat([horse_df, is_down], axis=1)

horse_df.fillna(value={'is_down': 0}, inplace=True)
horse_df['is_down'] = horse_df['is_down'].replace('(降)', 1)

## 余分な文字を削除
horse_df['rank'] = horse_df['rank'].apply(lambda x: x.replace("(降)", ""))
horse_df['rank'] = horse_df['rank'].apply(lambda x: x.replace("(再)", ""))

In [48]:
"""- 取・除はそもそも参加していないので削除
- 失は順位が全く当てにならないので情報を削除
- 中は最後まで到達していないが参加はしている。ひとまず20位にしておく"""

horse_df = horse_df[(horse_df['rank'] != "取") & (horse_df['rank'] != "除") & (horse_df['rank'] != "失")]
horse_df['rank'] = pd.DataFrame(horse_df['rank'].mask(horse_df['rank'] == "中", 20))

In [49]:
# 確認
horse_df["rank"].value_counts()

1     38027
2     38021
3     38012
5     38001
6     37983
4     37982
7     37882
8     37608
9     36989
10    35852
11    34199
12    32073
13    29079
14    25788
15    22141
16    16823
17     4096
18     3199
20     2752
Name: rank, dtype: int64

### 姓と年齢をsplit

In [50]:
horse_df['sex_and_age'].unique()

array(['牡4', '牡5', '牝4', '牝5', '牡6', '牡3', '牝3', '牡7', 'セ3', 'セ6', 'セ7',
       'セ8', '牡8', '牝7', '牝6', 'セ5', 'セ4', '牡10', 'セ9', '牡9', '牝8', '牡12',
       '牡11', 'セ10', '牝9', 'セ11', '牡2', '牝2', 'セ2', '牝10', '牡13'],
      dtype=object)

In [51]:
# 性別を別へ
"""
sex = horse_df["sex_and_age"].str.extract('(牝|牡|セ)', expand=True)
sex.columns ={"sex"}
horse_df = pd.concat([horse_df, sex], axis=1)

horse_df.fillna(value={'sex': 0}, inplace=True)
horse_df['sex'] = horse_df['sex'].replace('牝', 1)
horse_df['sex'] = horse_df['sex'].replace('牡', 2)
horse_df['sex'] = horse_df['sex'].replace('セ', 3)
## 余分な文字を削除
horse_df['sex_and_age'] = horse_df['sex_and_age'].str.strip("牝牡セ")
horse_df['sex_and_age'] = horse_df['sex_and_age'].astype(int)
"""

is_senba = horse_df["sex_and_age"].str.extract('(セ)', expand=True)
is_senba.columns ={"is_senba"}
horse_df = pd.concat([horse_df, is_senba], axis=1)

is_mesu = horse_df["sex_and_age"].str.extract('(牝)', expand=True)
is_mesu.columns ={"is_mesu"}
horse_df = pd.concat([horse_df, is_mesu], axis=1)

is_osu = horse_df["sex_and_age"].str.extract('(牡)', expand=True)
is_osu.columns ={"is_osu"}
horse_df = pd.concat([horse_df, is_osu], axis=1)


In [52]:
horse_df.fillna(value={'is_osu': 0}, inplace=True)
horse_df['is_osu'] = horse_df['is_osu'].replace('牡', 1)
horse_df.fillna(value={'is_mesu': 0}, inplace=True)
horse_df['is_mesu'] = horse_df['is_mesu'].replace('牝', 1)
horse_df.fillna(value={'is_senba': 0}, inplace=True)
horse_df['is_senba'] = horse_df['is_senba'].replace('セ', 1)
## 余分な文字を削除
horse_df['sex_and_age'] = horse_df['sex_and_age'].str.strip("牝牡セ")
horse_df['sex_and_age'] = horse_df['sex_and_age'].astype(int)

In [53]:
horse_df = horse_df.rename(columns={'sex_and_age': 'age'})

## goal_timeをtimedelta型にしてから秒に(last_timeも)

In [54]:
# nullになるのは、レースで「中」になった馬
print(horse_df['goal_time'].isnull().sum())
print(horse_df['last_time'].isnull().sum())

2752
2757


In [55]:
horse_df['goal_time'] = pd.to_datetime(horse_df['goal_time'], format='%M:%S.%f') - pd.to_datetime('00:00.0', format='%M:%S.%f')
horse_df['goal_time'] = horse_df['goal_time'].dt.total_seconds()

In [56]:
# 欠損値を最大値で埋める
horse_df.fillna(value={'goal_time': horse_df['goal_time'].max()}, inplace=True)
horse_df.fillna(value={'last_time': horse_df['last_time'].max()}, inplace=True)

In [57]:
horse_df.dtypes

race_id                  object
rank                     object
frame_number              int64
horse_number              int64
horse_id                 object
age                       int64
burden_weight           float64
rider_id                 object
goal_time               float64
half_way_rank            object
last_time               float64
odds                     object
popular                 float64
horse_weight             object
tamer_id                 object
owner_id                 object
date             datetime64[ns]
is_down                   int64
is_senba                  int64
is_mesu                   int64
is_osu                    int64
dtype: object

### goal_timeとレース距離から、平均速度を求める

In [58]:
# レース距離情報をmerge
race_tmp_df = race_df[["race_id", "distance"]]
horse_df = pd.merge(horse_df, race_tmp_df, on='race_id')

In [59]:
horse_df["distance"] = horse_df["distance"].astype(int)
horse_df["avg_velocity"] = horse_df["distance"]/horse_df["goal_time"]


### half_way_rank
splitして平均値を保持する（レースによってまちまちなので）

In [60]:
from statistics import mean
horse_df["half_way_rank"] = horse_df["half_way_rank"].apply(lambda x: mean([float(n) for n in (x.split("-"))]) if type(x) is str else float(x) )

In [61]:
horse_df[horse_df["rank"] == 20] = horse_df[horse_df["rank"] == 20].fillna({'half_way_rank': 20})
horse_df["half_way_rank"] = horse_df["half_way_rank"].fillna(horse_df['half_way_rank'].mean())
horse_df["half_way_rank"].isnull().sum()

0

In [62]:
horse_df["half_way_rank"] = horse_df["half_way_rank"].astype(float)

### horse_weight と diff の分離
「計不」は平均で穴埋め

In [63]:
horse_weight_dif = horse_df["horse_weight"].str.extract('\(([-|+]?\d*)\)', expand=True)
horse_weight_dif.columns ={"horse_weight_dif"}

horse_df = pd.concat([horse_df, horse_weight_dif], axis=1)

horse_df['horse_weight'] = horse_df['horse_weight'].replace('\(([-|+]?\d*)\)', '', regex=True)



In [64]:
horse_df['horse_weight'] = horse_df['horse_weight'].replace('計不', np.nan)
horse_df['horse_weight'] = horse_df['horse_weight'].astype(float)
horse_df['horse_weight_dif'] = horse_df['horse_weight_dif'].astype(float)

In [65]:
# 計不 の horse_idを探し、馬ごとの平均値で穴埋め
no_records = horse_df[horse_df['horse_weight'].isnull()]['horse_id']
for no_record_id in no_records:
    horse_df.loc[(horse_df['horse_id'] == no_record_id)&(horse_df['horse_weight'].isnull()), 'horse_weight'] = horse_df[horse_df['horse_id'] == no_record_id]['horse_weight'].mean() 
    horse_df.loc[(horse_df['horse_id'] == no_record_id)&(horse_df['horse_weight_dif'].isnull()), 'horse_weight_dif'] = 0 
    

In [66]:
horse_df.dtypes

race_id                     object
rank                        object
frame_number                 int64
horse_number                 int64
horse_id                    object
age                          int64
burden_weight              float64
rider_id                    object
goal_time                  float64
half_way_rank              float64
last_time                  float64
odds                        object
popular                    float64
horse_weight               float64
tamer_id                    object
owner_id                    object
date                datetime64[ns]
is_down                      int64
is_senba                     int64
is_mesu                      int64
is_osu                       int64
distance                     int64
avg_velocity               float64
horse_weight_dif           float64
dtype: object

### burden_weight, horse_weight の比率を追加

In [67]:
horse_df['burden_weight_rate'] = horse_df['burden_weight']/horse_df['horse_weight']

### last_time
とりあえず放置するが、外れ値の扱いを考えたほうが良さそう。

In [68]:
horse_df.plot(kind='hist', y='last_time' , bins=50, figsize=(16,4), alpha=0.5)

In [69]:
horse_df[horse_df['last_time']<20]['race_id'].unique()

array(['200808010804', '200806010208', '200806010304', ...,
       '201807040404', '201807040504', '201809050805'], dtype=object)

In [70]:
race_df[(race_df['race_id']=='200808010804') | (race_df['race_id']=='200806010208') | (race_df['race_id']=='200806010304')]

,race_id,race_round,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,horse_number_first,frame_number_second,horse_number_second,frame_number_third,horse_number_third,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3,is_obstacle,ground_type,is_left_right_straight,distance,weather_rain,weather_snow
49,200808010804,4,曇,4,2008-01-27 11:40:00,京都,14,7,12,1,1,8,13,380,130,110,430,350,340,180,1040,920,860,2020,0,1.0,芝,NaN,2930,0,0
64,200806010208,8,晴,1,2008-01-06 13:40:00,中山,14,6,9,5,7,2,2,2550,510,530,220,970,21030,4010,2050,1870,44390,33660,0,1.0,芝,NaN,3200,0,0
65,200806010304,4,雨,1,2008-01-12 11:20:00,中山,14,3,4,2,2,5,8,2340,330,150,150,2760,3300,880,850,290,8390,2610,0,1.0,芝,NaN,2880,2,0


### odds

In [71]:
horse_df['odds']= horse_df['odds'].astype(float)

### horse dataの保存

In [72]:
print(horse_df.dtypes)
horse_df.head(3)

race_id                       object
rank                          object
frame_number                   int64
horse_number                   int64
horse_id                      object
age                            int64
burden_weight                float64
rider_id                      object
goal_time                    float64
half_way_rank                float64
last_time                    float64
odds                         float64
popular                      float64
horse_weight                 float64
tamer_id                      object
owner_id                      object
date                  datetime64[ns]
is_down                        int64
is_senba                       int64
is_mesu                        int64
is_osu                         int64
distance                       int64
avg_velocity                 float64
horse_weight_dif             float64
burden_weight_rate           float64
dtype: object


,race_id,rank,frame_number,horse_number,horse_id,age,burden_weight,rider_id,goal_time,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,date,is_down,is_senba,is_mesu,is_osu,distance,avg_velocity,horse_weight_dif,burden_weight_rate
0,200810010312,1,4,7,2004110053,4,56.0,674,69.7,11.5,35.9,9.7,4.0,484.0,1075,19004,2008-01-26 16:05:00,0,0,0,1,1200,17.216643,-10.0,0.115702
1,200810010312,2,3,6,2003106038,5,57.0,1054,69.7,7.5,36.1,35.7,12.0,474.0,1007,910006,2008-01-26 16:05:00,0,0,0,1,1200,17.216643,-8.0,0.120253
2,200810010312,3,1,2,2004110268,4,53.0,5223,69.9,7.5,36.3,54.5,14.0,442.0,427,975009,2008-01-26 16:05:00,0,0,1,0,1200,17.167382,-18.0,0.119910


In [73]:
horse_df.to_csv("csv/cleaned_horse_data.csv", index=False )